In [ ]:
!pip install flask
!pip install pyngrok

In [ ]:
import os

# Create a directory named "templates" in the current working directory
os.makedirs("templates", exist_ok=True)

In [ ]:
from pyngrok import ngrok
import threading
import time
from flask import Flask

In [ ]:
from flask import Flask, render_template, request, jsonify
import joblib
import pandas as pd

# Set authentication token
ngrok.set_auth_token("2c54S0BCzBB0ZiGw5FSS82ulUNb_7wsNuyyPb22woziTCnnXn")

#Load pickel files
movies_dict  = joblib.load('movie_dict.pkl')
similarity = joblib.load('similarity.pkl')

# Start Flask app in a separate thread
def start_flask_app():
    app = Flask(__name__)

    @app.route('/')
    def index():
      movies = pd.DataFrame(movies_dict)
      return render_template('index.html',movies=movies['title'].values)

    @app.route('/recommend', methods=['POST'])
    def recommend():
      selected_movie = request.form.get('movie')
      recommendations = get_recommendations(selected_movie)
      return jsonify({'recommendations':recommendations})

    def get_recommendations(selected_movie):
      movies = pd.DataFrame(movies_dict)
      index = movies[movies['title'] == selected_movie].index[0]
      distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
      recommended_movie_names = []
      for i in distances[1:6]:
        movie_id = movies.iloc[i[0]].id
        recommended_movie_names.append(movies.iloc[i[0]].title)

      return recommended_movie_names


    app.run(port=5003)

flask_thread = threading.Thread(target=start_flask_app)
flask_thread.start()

# Wait for the Flask app to start
time.sleep(2)

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(addr='5003', proto='http', bind_tls=True)

# Print the URL
print(' * Tunnel URL:', ngrok_tunnel.public_url)

# Keep the Colab session alive
while True:
    time.sleep(3600)  # Keep the session alive for 1 hour.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5003
INFO:werkzeug:Press CTRL+C to quit


 * Tunnel URL: https://8414-34-73-174-74.ngrok-free.app


INFO:werkzeug:127.0.0.1 - - [17/Feb/2024 14:20:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2024 14:20:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2024 14:21:26] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2024 14:22:04] "POST /recommend HTTP/1.1" 200 -


KeyboardInterrupt: 